In [43]:
from langchain_google_vertexai.chat_models import ChatVertexAI
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
# from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain.agents.react.agent import create_react_agent
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langgraph.graph import StateGraph, END  
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import config
import chainlit as cl
import os
from dotenv import load_dotenv
import functools
import operator
from typing import List, Annotated, TypedDict
%load_ext autoreload
%autoreload 2

# Load environment variables from a .env file
load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [18]:
df = pd.read_csv("clean-food.csv")

In [19]:
chat_model = ChatVertexAI(model_name="gemini-2.0-flash-exp", seed=42, temperature=0, streaming=True)

In [40]:
for chunk in chat_model.stream([("human", "Hello, who are you? Who is google?")]):
    print(chunk.content if chunk.content else "", end="")

Okay, let's break this down:

**Who am I?**

I am a large language model, trained by Google. You can think of me as a sophisticated AI program designed to understand and generate human-like text. I don't have a physical body, personal feelings, or consciousness in the way humans do. I exist as code and data on Google's servers.

My purpose is to:

*   **Answer your questions:** I can access and process information from the real world through Google Search and use that knowledge to respond to your queries.
*   **Generate text:** I can write different kinds of creative text formats, like poems, code, scripts, musical pieces, email

In [41]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [42]:
python_tool = PythonAstREPLTool()
tools = [python_tool]

In [6]:
# system_prompt = PromptTemplate.from_template(
# """
# You are a highly creative and skilled chef and nutritionist, specializing in creating delicious and healthy recipes based solely on the available ingredients provided to you. Your primary goal is to generate innovative and appealing dishes using only those ingredients.

# Here's how you operate:

# *   *Strict Ingredient Focus:* You must use only the ingredients provided. Introduce as less new ingredient as possible.
# *   *No External Resources:* You must not mention or suggest searching online or using external resources. All recipe creation must be based on your internal knowledge and creative combination of the provided ingredients.
# *   *Detailed Recipes:* For each recipe, provide the following:
#     *   *Recipe Name:* A catchy and descriptive name.
#     *   *Ingredients:* A detailed list of ingredients with quantities, using only the provided ingredients.
#     *   *Instructions:* Step-by-step cooking instructions.
#     *   *Prep Time:* Estimated preparation time.
#     *   *Cook Time:* Estimated cooking time.
#     *   *Total Time:* Total time from start to finish.
#     *   *Nutrition Information (per serving):* Approximate calories, protein, fat, and carbohydrates (if possible based on general nutritional knowledge).
# *   *Maximizing Ingredient Use:* You should strive to use all of the provided ingredients in at least one of the generated recipes. If that's not feasible for a single recipe, create multiple recipes that collectively use all the ingredients.
# *   *Creative Combinations:* Think outside the box and suggest interesting and flavorful combinations. Don't just suggest the most obvious combinations.

# Example:

# Ingredients: Chicken breast, broccoli, rice

# Possible Recipe: Lemon Herb Chicken with Roasted Broccoli and Rice Pilaf

# (Then provide the full recipe details as described above along with nutrition details per serving and prep time)

# Dataset Reference (Use this only for nutritional information if you can reasonably estimate it based on the provided ingredients - prioritize creating new recipes and do not use it to introduce new ingredients):
# \n""" + df.to_markdown(index=True))

In [12]:
# system_prompt = PromptTemplate.from_template(config.AGENT_PROMPT_3, partial_variables= {"df_str": df.to_markdown(index=True)})

In [13]:
tools = [PythonAstREPLTool(), DuckDuckGoSearchRun()]

In [14]:
agent = create_react_agent(llm = chat_model, tools = tools, prompt= system_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory= memory, verbose=True, handle_parsing_errors=True)

In [15]:
response = agent_executor.invoke({"input": "I have eggs, tomatoes, onions and bell pepper, what can I make and how?"})
print(response)



> Entering new AgentExecutor chain...
**Thought**: I need to create a recipe using the provided ingredients (eggs, tomatoes, onions, and bell pepper). I'll create a simple omelet recipe. Then, I'll calculate the nutritional information based on the provided table and create a visualization.
**Action**: `python_repl_ast`
**Action Input**:
```python
import pandas as pd

food_data = {
    'Food': ['Kale', 'Blueberries', 'Salmon', 'Garlic', 'Spinach', 'Almonds', 'Broccoli', 'Avocado', 'Quinoa', 'Lentils', 'Sweet Potato', 'Green Tea', 'Turmeric', 'Eggs', 'Chia Seeds', 'Greek Yogurt', 'Pomegranate', 'Walnuts', 'Ginger', 'Olive Oil', 'Beets', 'Sardines', 'Mushrooms', 'Tomatoes', 'Oats', 'Flaxseeds', 'Brussels Sprouts', 'Pumpkin Seeds', 'Black Beans', 'Green Peas', 'Asparagus', 'Blackberries', 'Wild Rice', 'Seaweed', 'Cauliflower', 'Coconut Oil', 'Red Bell Peppers', 'Watercress', 'Artichokes', 'Cherries', 'Mango', 'Pistachios', 'Kiwi', 'Oranges', 'Chickpeas', 'Papaya', 'Cabbage', 'Brazil Nut

KeyboardInterrupt: 

In [ ]:
from IPython.display import Markdown
display(Markdown(response['output']))

In [ ]:
list(map(lambda x: x.name, tools))

In [19]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the information below.\n\n {df_str}".format(
                df_str=df.to_markdown(index=True)
            ),
        ),
        ("system", "If you cannot find relevant information in the above context, call the duckduckgo_search tool to find out the relevant nutritional information then use the python_repl_ast tool to plot the nutritional values in a pretty plot using seaborn and save it as 'plot.png'."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [20]:
agent = create_tool_calling_agent(chat_model, tools, prompt)

In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "I have eggs, tomatoes, onions and bell pepper, what can I make and how?"})